In [1]:
# Naive Bayes without library

from __future__ import unicode_literals

import re
import string

import numpy as np
import pandas as pd

from hazm import *
from cleantext import clean

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df['comment'] = train_df['comment'].fillna('XXXXXXXX')
test_df['comment'] = test_df['comment'].fillna('XXXXXXXX')

In [21]:
def word_extraction(sentence):
    normalizer = Normalizer()
    sentence = normalizer.normalize(sentence)
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(sentence)
    words = [w for w in words if len(w) > 1 ]
    words = [w.lower() for w in words]
    # words = [re.split(r'([a-zA-Z]+)', w) for w in words]
    cleaned_words = []
    for word in words:
        cleaned_words_temp = re.split(r'([a-zA-Z]+)', word)
        cleaned_words.extend(word for word in cleaned_words_temp if len(word) >= 1)
    return cleaned_words

In [9]:
def tokenize(sentences):
    words = []
    for sentence in sentences:
        # print(sentence)
        w = word_extraction(sentence)
        words.extend(w)
    words = sorted(list(set(words)))
    return words

In [16]:

comments_ham = train_df.query('verification_status==1')['comment'].values.tolist()
comments_spam = train_df.query('verification_status==0')['comment'].values.tolist()

all_comments = []
all_comments.extend(comments_spam)
all_comments.extend(comments_ham)

vocabulary = tokenize(all_sentences)
